In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp metrics

# Metrics

> Metrics and norms

In [ ]:
#export 
from nbdev.showdoc import *
from grapht.graphtools import laplacian
from functools import lru_cache
import networkx as nx
import numpy as np
import scipy.sparse as sp

## Norms

Scipy doesnt have implementations for these norms 

In [ ]:
#export 
def sparse_2norm(A):
    "Returns the matrix 2-norm of a sparse matrix `A`"
    return np.abs(sp.linalg.eigsh(A, k=1, which='LM', return_eigenvectors=False))[0]

def sparse_maxnorm(A):
    "Returns the max |A_ij| for a sparse matrix `A`"
    return max(-A.min(), A.max())

## Metrics

The Laplacian distance isnt a real metric but 🤷‍♂️

In [ ]:
#export
def laplacian_distance(G, Gp, setdiag=False):
    "Calculates $|| \mathcal{L} -  \mathcal{L}_p ||$ using the matrix 2-norm"
    L = laplacian(G, setdiag)
    Lp = laplacian(Gp, setdiag)
    E = Lp - L
    return sparse_2norm(E)

## Line distances

This is used to calculate the distance between edges. The distances are cached

In [ ]:
#export
class LineDistances():
    
    def __init__(self, G):
        self.G = G
        self.line_graph = nx.line_graph(G)
        
    @lru_cache(maxsize=None)
    def __call__(self, edge1, edge2):
        return nx.shortest_path_length(self.line_graph, edge1, edge2)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_metrics.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.
